Import packages

In [1]:
# import sys, os
# import pathlib
# # __location__ = os.getcwd()
# # __location__ = pathlib.Path(__location__)

# # sys.path.append('C:\\Users\\doore\\project\\snake_RL\\dmc\\domains')
# import snake
# import snake_v2
# import snake_v3


# from snake.envs.SnakeEnv import SnakeEnv # for ray env register
# from snake_v2.envs.SnakeEnv import SnakeEnv # for ray env register
from snake_v3.envs.SnakeEnv import SnakeEnv # for ray env register
import gymnasium as gym

# import rl 알고리즘
from ray.rllib.algorithms.ppo import PPOConfig 
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from ray.rllib.policy.policy import Policy

from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

import numpy as np

c:\Users\doore\anaconda3\envs\rllib\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Make Snake Env

In [2]:
env = gym.make('snake/SnakeEnv-v3', render_mode="human")

register_env("snake-v3", lambda config: SnakeEnv())


Load pre-learned policy

In [3]:
from ray.rllib.models import ModelCatalog
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.preprocessors import get_preprocessor
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.models.torch.recurrent_net import RecurrentNetwork as TorchRNN
from ray.rllib.models.tf.recurrent_net import RecurrentNetwork
from ray.rllib.utils.annotations import override
from ray.rllib.utils.framework import try_import_tf, try_import_torch

policy = Policy.from_checkpoint('C:/Users/doore/ray_results/PPO_snake-v3_2023-03-15_14-35-41vcm0unve/checkpoint_000581')



In [4]:
## For Recurrent Policy

lstm_dim = 32
num_lstm_cells = 2
initial_state = np.zeros((num_lstm_cells, lstm_dim))

_state = initial_state
_obs, _ = env.reset()

for epi in range(10):
    for i in range(10):
        _act, _state, _ = policy['default_policy'].compute_single_action(obs=_obs, state=_state)
        _obs, _reward, _, _, _dict = env.step(_act[0])
        print(_dict['forward_reward'])
    _obs, _ = env.reset()
env.close()

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
policy['default_policy'].model.get_parameter

Do simulation

In [ ]:
_obs, _ = env.reset()

c_reward = 0
for epi in range(10):
    for i in range(400):
        _act = policy['default_policy'].compute_single_action(_obs)
        _obs, _reward, _, _, _ = env.step(_act[0])
        c_reward = c_reward + _reward
    _obs, _ = env.reset()
    print(c_reward)
    c_reward = 0
    
env.close()